TODOs:
- Update the code with the proper example (HF or Ne)
- Move QEMIST Cloud code at the end in an extra section?
- Concluding remarks
- Proper review

# Importing MI-FNO fragments from QEMIST Cloud

The method of increment (MI) approach expresses the electron correlation energy of a molecular system as a truncated many-body expansion in terms of orbitals, atoms, molecules, or fragments. The electron correlation of the system is expanded in terms of occupied orbitals, and the MI approach is employed to systematically reduce the occupied orbital space. At the same time, the virtual orbital space is reduced based on the frozen natural orbitals (FNO), which are obtained using a one-particle density matrix from second-order, many-body perturbation theory. In this way, a method referred to as the MI-FNO approach is available for the systematic reduction of both the occupied space and the virtual space in quantum chemistry simulations. 

The MI method, first introduced in quantum chemistry by Nesbet ([Phys. Rev. 1967, 155, 51](https://doi.org/10.1103/PhysRev.155.51), [Phys. Rev. 1967, 155, 56](https://doi.org/10.1103/PhysRev.155.56) and [Phys. Rev. 1968, 175, 2](https://doi.org/10.1103/PhysRev.175.2)), is based upon the n-body Bethe–Goldstone expansion ([Proc. R. Soc. A, 1957, 238, 551](https://doi.org/10.1098/rspa.1957.0017)) of the correlation energy of a molecule. The correlation energy ($E_c$), defined as the difference between the exact ($E_{\text{exact}}$) and the Hartree–Fock (mean-field) energy ($E_{\text{HF}}$), can be expanded as

$$
\begin{align*}
E_c &= E_{\text{exact}} - E_{\text{HF}} \\
&= \sum_i \epsilon_i + \sum_{i>j} \epsilon_{ij} + \sum_{i>j>k} \epsilon_{ijk} + \sum_{i>j>k>l} \epsilon_{ijkl} + \dots
\end{align*}
$$

where $\epsilon_i$, $\epsilon_{ij}$, $\epsilon_{ijk}$, and $\epsilon_{ijkl}$ are, respectively, the one-, two-, three-, and four-body increments (expansions) defined as

$$
\begin{align*}
\epsilon_i &= E_c(i) \\
\epsilon_{ij} &= E_c(ij) - \epsilon_i - \epsilon_j \\
\epsilon_{ijk} &= E_c(ijk) - \epsilon_{ij} - \epsilon_{ik} - \epsilon_{jk} - \epsilon_{i} - \epsilon_{j} - \epsilon_{k} \\
\epsilon_{ijkl} &= E_c(ijkl) - \epsilon_{ijk} - \epsilon_{ijl} - \epsilon_{jkl} - \dots \\
&\vdots
\end{align*}
$$

The following figure, taken from [J. Chem. Phys. 2021, 155, 034110](https://doi.org/10.1063/5.0054647), illustrates this problem decomposition scheme in terms of 1-body and many-body interactions. On each subproblem, an FNO truncation
is applied to reduce their virtual space. The subproblems resulting from the MI-FNO reduction can then be solved by any algorithm, including quantum algorithms such as the phase estimation algorithm and the variational quantum eigensolver, to predict the correlation energies of a molecular system.

<div>
<img src="img/mifno.png" width="600"/>
</div>

This problem decomposition pipeline is available in QEMIST Cloud. Here in this notebook, we will go through the steps of how to export MI-FNO fragment data and import them in Tangelo for further treatment.

## System definition

We will take as an exemple the hydrogen fluoride (HF) system in the `cc-pvdz` basis.

In [6]:
HF_coordinates = """
    F    0.0000   0.0000   0.0000
    H    0.0000   0.0000   0.9168
"""
basis = "cc-pvdz"
charge = 0 
spin = 0

Nextly, we will demonstrate the pipeline from start to finish with a working python environment with both `qemist_client` and `tangelo` installed.

## MI-FNO calculation

With the `qemist_client` python module, it is possible to send conventional quantum chemistry problem to the QEMIST Cloud infrastructure. [QEMIST Cloud](https://goodchemistry.com/qemist-cloud/), as stated on the product webpage, is an engine that enables faster, more accurate, and scalable ways to perform computational chemistry simulations. This platform leverages easily and readily accessible computers on the cloud to perform chemistry simulations that were previously intractable even on expensive, high-performance computing environments. 

We do not assume that you have `qemist_client` installed, so we provide a code snipplet used to generate the problem decomposition results. Each fragment's virtual space has been truncated to keep only one virtual orbital with the highest occupation number.
```python
import os
os.environ['QEMIST_PROJECT_ID'] = "your_project_id_string"
os.environ['QEMIST_AUTH_TOKEN'] = "your_qemist_authentication_token"

import qemist_client
from qemist_client.problem_decomposition import IncrementalDecomposition
from qemist_client.util.natural_orbitals import TruncationMethod
from qemist_client.problem_reduction import FNO
from qemist_client.electronic_structure_solvers import HBCI

HF_coordinates = """
    F    0.0000   0.0000   0.0000
    H    0.0000   0.0000   0.9168
"""
basis = "cc-pvdz"
charge = 0
spin = 0

# Creating the QEMIST molecule object.
HF_mol = qemist_client.molecule.Molecule(HF_coordinates, basis=basis, charge=charge, spin=spin)

# Defining the solvers, Heath-Bath Configuration Interaction (HBCI),
# Frozen Natural Orbitals (FNO), and the problem decomposition approach,
# Method of Increments (MI).
hbci_solver = HBCI()
fno = FNO(hbci_solver, 
          export_fragment_data=True, 
          method_of_truncation=TruncationMethod.COUNT, 
          method_of_truncation_threshold=1)
mi_solver = IncrementalDecomposition(solver=fno,
                                     truncation_order=2)
# Submitting the problem to the cloud.
HF_handle = mi_solver.simulate(molecule=HF_mol)
print(f"\nHF handle: {HF_handle}")

# Retrieving the results.
HF_result = mi_solver.get_result()
print(HF_result)
```

For more information about the `qemist_client` API and QEMIST Cloud feature set, we refer the reader to the [QEMIST Cloud documentation](https://alpha-api.qemist.cloud/#/docs). The `HF_result` python dictionary can be saved in a json file (for further treatment in `tangelo` or for archiving purposes) with the following code.
```python
import json
with open("your_file_path", "w") as f:
    json.dump(HF_result, f)
```

## MI-FNO fragments into Tangelo

### Importation

In the `tangelo` package, the `MIFNOHelper` serves the function of importing the `HF_result` generated from QEMIST Cloud.

In [8]:
from tangelo.problem_decomposition import MIFNOHelper

For this example, the `HF_results` has been saved in `./data/HF_MIFNO_trunc2.json`. During the runtime of the QEMIST Cloud script, a user could have called `MIFNOHelper(results_object=HF_result)` and it would have ended with the same results.

In [33]:
fno_fragments = MIFNOHelper(json_file="./data/HF_MIFNO_trunc2.json")
print(fno_fragments)

(All the energy values are in hartree)
Total MI-FNO energy = -100.2187315187088
Correlation energy = -0.19931281562172387
Mean-field energy = -100.01941870308707
        correction                          frozen_orbitals_truncated  \
(1,)     -0.006195  [0, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...   
(2,)     -0.007127  [0, 1, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...   
(3,)     -0.008654  [0, 1, 2, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...   
(4,)     -0.008654  [0, 1, 2, 3, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...   
(1, 2)   -0.030123  [0, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...   
(1, 3)   -0.040116  [0, 2, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...   
(1, 4)   -0.040116  [0, 2, 3, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...   
(2, 3)   -0.048697  [0, 1, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...   
(2, 4)   -0.048697  [0, 1, 3, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...   
(3, 4)   -0.056031  [0, 1, 2, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...   

         epsilon  energy_correlati

### Downloading molecular coefficients files

In [15]:
# Downloading and/or reading the HDF5 files in the data folder.
# If a download request is made, a message is printed.
fno_fragments.retrieve_mo_coeff("./data")

### Reconstructing a fragment Hamiltonian

In [19]:
from tangelo import SecondQuantizedMolecule
from tangelo.toolboxes.operators import count_qubits

mol = SecondQuantizedMolecule(HF_coordinates, q=charge, spin=spin, basis=basis)

selected_fragment = "(3, 4)"

ferm_op = fno_fragments.compute_fermionoperator(mol, selected_fragment)
qu_op = fermion_to_qubit_mapping(ferm_op, mapping="JW")

print(f"The qubit operator for the {selected_fragment} fragment is mapped to {count_qubits(qu_op)} qubits.")

The qubit operator for the (3, 4) fragment is mapped to 6 qubits.


### QITE on a Fragment

In [29]:
from tangelo.algorithms.projective import QITESolver
from tangelo.toolboxes.operators import qubitop_to_qubitham

qubit_ham = qubitop_to_qubitham(qu_op, mapping="JW", up_then_down=False)

qite = QITESolver({"qubit_hamiltonian": qubit_ham, "n_electrons": 4, "n_spinorbitals": 6})
qite.build()
qite.simulate()

Iteration 1 of QITE with starting energy -100.01941870308727
Iteration 2 of QITE with starting energy -100.02286250399173
Iteration 3 of QITE with starting energy -100.02520229253774
Iteration 4 of QITE with starting energy -100.02679118526885
Iteration 5 of QITE with starting energy -100.0278699048825
Iteration 6 of QITE with starting energy -100.02860223507179
Iteration 7 of QITE with starting energy -100.02909946796684
Iteration 8 of QITE with starting energy -100.0294371651163
Iteration 9 of QITE with starting energy -100.0296666029956
Iteration 10 of QITE with starting energy -100.02982256841632
Iteration 11 of QITE with starting energy -100.02992865864991
Iteration 12 of QITE with starting energy -100.03000088085165
Iteration 13 of QITE with starting energy -100.03005009467229
Iteration 14 of QITE with starting energy -100.03008366905831
Iteration 15 of QITE with starting energy -100.03010660572828
Iteration 16 of QITE with starting energy -100.03012230089017
Iteration 17 of QITE

-100.03015714656397

### Summation of the incremental energy
- Mp2 correction automatically added.

In [30]:
e = fno_fragments.mi_summation({"(3, 4)": qite.final_energy})
print(e)

-100.21873123150789


In [32]:
print(fno_fragments.e_tot)

-100.2187315187088


## Closing words

- Combiner PD at scale dans le cloud (professionnel), avec tangelo, regarder l'impact du calcul quantique sur des plus gros problèmes (plus gros use case).
- MI-FNO fragment, see the impact on the incremental sum
- The algo is the user choice! (custom?!)